In [28]:
!pip install pandas scikit-learn nltk

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

data = {
    'text': ['I love this movie', 'This is the worst film ever', 'It was an okay experience', 'Excellent performance!', 'Not bad, but could be better.'],
    'sentiment': ['positive', 'negative', 'neutral', 'positive', 'neutral']
}
df = pd.DataFrame(data)

print(df.head())

                            text sentiment
0              I love this movie  positive
1    This is the worst film ever  negative
2      It was an okay experience   neutral
3         Excellent performance!  positive
4  Not bad, but could be better.   neutral


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'sentiment140' dataset.
Path to dataset files: /kaggle/input/sentiment140


In [31]:
!pip install -q kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"sirmazen","key":"711b1b7ade65b701436a7142cc63bc3e"}'}

In [32]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [33]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [34]:
import zipfile
import os
with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [35]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header=None)
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
print(df.head())

   sentiment          id                          date     query  \
0          0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1          0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2          0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3          0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4          0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [36]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

import nltk
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()

    text = re.sub('https?://\S+|www\.\S+', '', text)

    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]

    return " ".join(filtered_words)

df['text'] = df['text'].apply(clean_text)

print(df[['text', 'sentiment']].head())

<>:15: SyntaxWarning: invalid escape sequence '\S'
<>:20: SyntaxWarning: invalid escape sequence '\w'
<>:15: SyntaxWarning: invalid escape sequence '\S'
<>:20: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-2872816674.py:15: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
/tmp/ipython-input-2872816674.py:20: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  sentiment
0  switchfoot awww thats bummer shoulda got david...          0
1  upset cant update facebook texting might cry r...          0
2  kenichan dived many times ball managed save re...          0
3                   whole body feels itchy like fire          0
4           nationwideclass behaving im mad cant see          0


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')




X = df['text']
y = df['sentiment'].map({0: 0, 4: 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vectorized, y_train)


def clean_text(text):
    text = text.lower()

    text = re.sub('https?://\S+|www\.\S+', '', text)

    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]

    return " ".join(filtered_words)


def predict_sentiment(text):
    # 1. Clean the text using the function we defined earlier
    cleaned_text = clean_text(text)

    # 2. Vectorize the cleaned text using the *fitted* vectorizer
    text_vector = vectorizer.transform([cleaned_text])

    # 3. Predict the sentiment
    prediction = model.predict(text_vector)[0]

    # 4. Map the prediction to a readable sentiment label
    # Note: Our model is trained on 0 (negative) and 1 (positive)
    sentiment_map = {1: 'positive', 0: 'negative'}
    predicted_sentiment = sentiment_map[prediction]

    return predicted_sentiment

# Test the function with new sentences
test_sentences = [
    "I am very happy with this project!",
    "This is a terrible idea.",
    "The weather is just fine." # Note: Neutral sentiment might not be predicted accurately as the model is trained on binary sentiment.
]

for sentence in test_sentences:
    sentiment = predict_sentiment(sentence)
    print(f"Sentence: '{sentence}' -> Predicted Sentiment: '{sentiment}'")

<>:47: SyntaxWarning: invalid escape sequence '\S'
<>:52: SyntaxWarning: invalid escape sequence '\w'
<>:47: SyntaxWarning: invalid escape sequence '\S'
<>:52: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-2101944004.py:47: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
/tmp/ipython-input-2101944004.py:52: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


Sentence: 'I am very happy with this project!' -> Predicted Sentiment: 'positive'
Sentence: 'This is a terrible idea.' -> Predicted Sentiment: 'negative'
Sentence: 'The weather is just fine.' -> Predicted Sentiment: 'positive'
